### Term 1 Project 3 : Behavioral Cloning
#### Project Writeup

##### Behavioral Cloning Project

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report
---
#### Rubric Points
Here I will consider the [rubric points](https://review.udacity.com/#!/rubrics/432/view) individually and describe how I addressed each point in my implementation

---
### Files Submitted & Code Quality

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:
* model.py - Contains the script for the model, pre-processing and augmentation of data, training
* drive.py - Used for driving the car in autonomous mode using the model generated by model.py
* model.h5 - Contains a trained convolution neural network 
* video.mp4 - Autonomous run in the simulator for Track 1 based on the model generated by model.py
* writeup_report.md - Contains project writeup

#### 2. Submission includes functional code
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```

#### 3. Submission code is usable and readable

The model.py file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.

---
### Model Architecture and Training Strategy

#### 1. An appropriate model architecture has been employed
My model is based on the NVIDIA End-End Deep Learning for Autonomous Cars. I've tweaked the architecture to fit to my purpose.

A pictorial representation of the model can be seen below

![Behavioral Cloning Model Architecture] (ModelArchitecture.jpg)

This consists of several layers of Convolutional and Fully Connected layers.
Here's my model architecture with the hyperparameters in each layer